In [59]:
#pip install librosa 

In [60]:
import librosa

In [61]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [76]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [63]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [95]:
emotions['04']

'sad'

In [79]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size):
    x,y=[],[]

    for file in glob.glob("C:\\Users\\sachin.bhoi\\Documents\\Jupyter_Notebook\\Speech_emotion_recognition\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

#"D:\\DataFlair\\ravdess data\\Actor_*\\*.wav"
#C:\Users\sachin.bhoi\Documents\Jupyter_Notebook\Speech_emotion_recognition\speech-emotion-recognition-ravdess-data


In [80]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

array([[-6.01416077e+02,  6.33363037e+01, -1.16155634e+01, ...,
         7.30560214e-06,  4.64440200e-06,  1.78248013e-06],
       [-7.87200195e+02,  5.97919426e+01,  2.11022396e+01, ...,
         4.30602057e-07,  4.01216226e-07,  4.06060309e-07],
       [-5.60616211e+02,  5.78973503e+01, -8.86894703e+00, ...,
         6.46989676e-04,  3.31015413e-04,  1.85833647e-04],
       ...,
       [-4.44761963e+02,  2.36135769e+01, -1.48351011e+01, ...,
         8.65290698e-04,  5.16015454e-04,  3.73600313e-04],
       [-5.96088257e+02,  6.75587769e+01,  1.62860453e+00, ...,
         2.92377263e-05,  2.20639995e-05,  1.13877868e-05],
       [-6.95250610e+02,  4.46737938e+01, -7.46818161e+00, ...,
         2.15334803e-05,  1.84785295e-05,  1.01300748e-05]])

In [81]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [82]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [83]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [84]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [85]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)
y_pred

array(['happy', 'calm', 'happy', 'happy', 'calm', 'calm', 'calm', 'happy',
       'calm', 'happy', 'happy', 'fearful', 'fearful', 'happy', 'disgust',
       'happy', 'calm', 'calm', 'happy', 'calm', 'calm', 'disgust',
       'calm', 'calm', 'happy', 'happy', 'calm', 'happy', 'calm',
       'fearful', 'happy', 'happy', 'happy', 'fearful', 'happy', 'calm',
       'calm', 'fearful', 'calm', 'calm', 'happy', 'calm', 'fearful',
       'calm', 'fearful', 'calm', 'calm', 'calm', 'calm', 'happy',
       'fearful', 'fearful', 'fearful', 'happy', 'happy', 'happy', 'calm',
       'happy', 'calm', 'calm', 'disgust', 'calm', 'happy', 'calm',
       'happy', 'calm', 'calm', 'calm', 'fearful', 'happy', 'fearful',
       'fearful', 'fearful', 'fearful', 'fearful', 'disgust', 'fearful',
       'happy', 'calm', 'fearful', 'calm', 'calm', 'fearful', 'calm',
       'disgust', 'calm', 'calm', 'fearful', 'happy', 'happy', 'happy',
       'fearful', 'calm', 'happy', 'calm', 'disgust', 'calm', 'calm',
       

In [86]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 60.42%
